***Импорт библиотек***

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random

***Подтягиваем файлы***

In [7]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [8]:
def custom_StandardScaler(x):
    res = (x - x.mean()) / x.std()
    return res

In [9]:
def pipl(df1):
    df=df1.copy()
    df['count_subj']=df['physics']+df['biology']+df['chemistry']+df['english']+df['geography']+df['history']
    df['s_lesson_price']=custom_StandardScaler(df['lesson_price'])
    df['s_mean_exam_points']=custom_StandardScaler(df['mean_exam_points'])
    df['s_age']=custom_StandardScaler(df['age'])
    
    df=df.drop(['age','lesson_price','mean_exam_points','physics','biology','chemistry','english','geography','history'],axis=1)
    try:
        df2=df[[ 'count_subj','s_lesson_price', 's_mean_exam_points', 's_age','choose']]
    except:
        df2=df[[ 'count_subj','s_lesson_price', 's_mean_exam_points', 's_age']]
    return df2


In [10]:
df=train.pipe(pipl)
df.head()

,count_subj,s_lesson_price,s_mean_exam_points,s_age,choose
0,0.0,0.854466,0.703552,-1.345023,0
1,3.0,-0.863783,-0.546906,0.771966,1
2,1.0,0.090800,0.115101,-2.092195,0
3,0.0,-1.245616,0.115101,-1.594080,1
4,1.0,1.045383,0.629995,0.024794,0


In [11]:
train_data=df.to_numpy()

In [38]:
x = train_data[:,:-1]

y = train_data[:, -1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

print(train_data[np.where(train_data[:, -1]== 1)].shape)
print(train_data[np.where(train_data[:, -1]== 0)].shape)

(1109, 5)
(8891, 5)


***Прописываем/копируем миллиард самописных функции***

In [13]:
def get_bootstrap(data, labels, n):
    n_samples = data.shape[0]
    bootstrap = []

    # by rows in the sample
    for i in range(n):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        # by columns in the sample 
        for j in range(n_samples):
            # shuffling index by random
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
        
        bootstrap.append((b_data, b_labels))

    return bootstrap

def get_subsample(len_sample):
    # list of indixes from zero to len
    sample_indices = [i for i in range(len_sample)]
    # length of subsample as sqr root of the whole length
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    # shuffling the indices of the sample
    random.shuffle(sample_indices)

    for i in range(len_subsample):
        subsample.append(sample_indices.pop())

    return subsample

class Node:
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index # index of the feature comapred to threshold
        self.t = t # threshold value
        self.true_branch = true_branch # subsample in compliance with the condition in the node
        self.false_branch = false_branch # subsample not in compliance with the condition in the node

# base case node == leaf
class Leaf:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()

    def predict(self):
        # qaunt of the objects of various classes
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        prediction = max(classes, key=classes.get)

        return prediction

def gini(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2

    return impurity

# calc the quality 
def quality(left_labels, right_labels, current_gini):
    # samples elements in the left subtree
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

# splitting dataset in the node
def split(data, labels, index, t):

    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)

    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]

    return true_data, false_data, true_labels, false_labels

def find_best_split(data, labels):

    # min quant of objects in the node
    min_leaf = 1

    current_gini = gini(labels)

    best_quality, best_t, best_index = 0, None, None

    n_features = data.shape[1]

    # subsample size = sq root of the sample size
    subsample = get_subsample(n_features)

    for index in subsample:
        # getting only unique values of a feature
        t_values = np.unique([row[index] for row in data])

        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            # skipping splits with less than 5 objects within
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)

            # getting threshold with the max increase in quality
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

def build_tree(data, labels):
    quality, t, index = find_best_split(data, labels)

    # base case if not quality increase
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # building true & false subtrees recursively
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    return Node(index, t, true_branch, false_branch)

# building random forest
def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)

    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels))

    return forest

# classification function
def classify_object(obj, node):

    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

# prediction for the sample by one tree
def predict(data, tree):

    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)

    return classes

# prediction by trees' vote
def tree_vote(forest, data):

    # all prediction in a list
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))

    # list with predictions for the each of the objects
    predictions_per_object = list(zip(*predictions))

    # the preidction with the most of votes
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))

    return voted_predictions

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


***Прогоняем случайный лес с разным количеством деревьев***

In [14]:
# testing various quantity of trees
trees_quant = [5, 7, 10, 13]
for tree_numb in trees_quant:
    my_model = random_forest(x_train, y_train, tree_numb)
    train_answers = tree_vote(my_model, x_train)
    print(tree_numb,':')
    print(f'accuracy @train: {accuracy_metric(y_test, train_answers)}')
    y_pred = tree_vote(my_model, x_test)
    print(f'accuracy @test: {accuracy_metric(y_test, y_pred)}')

5 :
accuracy @train: 82.8
accuracy @test: 87.3
7 :
accuracy @train: 83.2
accuracy @test: 87.6
10 :
accuracy @train: 84.05
accuracy @test: 88.44999999999999
13 :
accuracy @train: 82.95
accuracy @test: 88.44999999999999


***Пропишем в модели количество деревьев c найлучшим показателем***

In [16]:
# predicting with the optimal number of tree
my_model = random_forest(x, y, 10)

train_answers = tree_vote(my_model, x_train)
print(f'@test: {accuracy_metric(y_test, y_pred)}')

@test: 88.44999999999999


***Сделаем предсказание и сохраним данные в файл***

In [17]:
df1=test.pipe(pipl)
df1.head()

,count_subj,s_lesson_price,s_mean_exam_points,s_age
0,0.0,1.900372,1.904968,-1.733633
1,2.0,0.190191,0.504831,-1.360126
2,0.0,-0.949930,-1.411147,-0.239605
3,1.0,2.375422,2.052351,-0.239605
4,1.0,-0.569889,-0.453158,-0.986619


In [18]:
df1.shape

(10000, 4)

In [19]:
test_data=df1.to_numpy()

In [20]:
test_data.shape

(10000, 4)

In [21]:
y_pred = tree_vote(my_model, test_data)

In [22]:
np.savetxt(
    'submission.csv',
    np.column_stack((np.array(range(10000,20000)), np.array(y_pred))),
    delimiter=',',
    header='Id,choose',
    comments='',
    fmt='%d'
)

***Данная модель показала результат 0.57***

***Попробуем сбалансировать данные, чтобы к модели подавались классы без сильного искажения***

In [35]:
# балансировка
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys

balanced_samples = balanced_subsample(x_train, y_train)

print(f'positive target: {len(np.where(balanced_samples[-1] == 1)[0])}')
print(f'negative target: {len(np.where(balanced_samples[-1] == 0)[0])}')

x_train = balanced_samples[0]
y_train = balanced_samples[1]

positive target: 878
negative target: 878


***Пропишем в модели количество деревьев с найлучшим показателем***

In [24]:
# predicting with the optimal number of tree
my_model = random_forest(x_train, y_train,10)

print(f'@test: {accuracy_metric(y_test, y_pred)}')

@test: 85.2


***Сделаем предсказание и сохраним данные в файл***

In [25]:
y_pred = tree_vote(my_model, test_data)

In [26]:
np.savetxt(
    'submission.csv',
    np.column_stack((np.array(range(10000,20000)), np.array(y_pred))),
    delimiter=',',
    header='Id,choose',
    comments='',
    fmt='%d'
)

***Результат в соревновании 0.73306***